In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

data = pd.read_csv('data/dataset_reduced.csv')

data['target_class'] = pd.qcut(data['Cs'], q=10, labels=False)
X = data.drop(['Cs', 'target_class'], axis=1)
y = data['Cs']
stratify_column = data['target_class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=21, stratify=stratify_column)

X_train = pd.DataFrame(X_train, columns=X.columns)
X_test = pd.DataFrame(X_test, columns=X.columns)

X_train

In [2]:
statistics = X_train.describe(include='all')
statistics

In [3]:
median = statistics.iloc[5]
median

In [4]:
import pandas as pd
import numpy as np

# 固定列的值
fixed_values = {
    "SSA": 1667.3,
    "PV": 1.2,
    "Dap": 2.508,
    "ID/IG": 1.02,
    "Mic": 0.48,
    "CD": 0.2
}

# 生成等距分布的Dap和ID/IG的值
N_values = np.linspace(1, 20, 100)
O_values = np.linspace(1, 30, 100)

columns = [
    "O", "N", "SSA", "PV",  "Mic","Dap", "ID/IG", 
     "Current density",
]

# 创建一个空的DataFrame
data = []

# 填充数据
for O in O_values:
    for N in N_values:
        row = fixed_values.copy()
        row["N"] = N
        row["O"] = O
        data.append(row)

df = pd.DataFrame(data, columns=columns)
df

In [5]:
from xgboost import XGBRegressor

xgb = XGBRegressor(n_estimators=80,
                             learning_rate=0.14,
                             subsample=0.5,
                             gamma=0.1,
                             max_depth=8,
                             min_child_weight=2,
                             reg_alpha=0.26,
                             colsample_bytree=0.7,
                             colsample_bylevel=1.0,
                             colsample_bynode=0.6,
                             random_state=21)
xgb.fit(X_train, y_train)

y_train_pred = xgb.predict(X_train)
y_test_pred = xgb.predict(X_test)
y_generated_pred = xgb.predict(df)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import griddata

# 生成示例数据
# 这部分需要替换为你的实际数据
feature_1 = df['N']
feature_2 = df['O']
c = y_generated_pred

# 创建网格数据
x = np.linspace(1, 20, 10)
y = np.linspace(1, 30, 10)
X, Y = np.meshgrid(x, y)

# 插值
Z = griddata((feature_1, feature_2), c, (X, Y), method='cubic')

# 绘制等高线图
plt.figure(figsize=(20, 19))
plt.rcParams.update({
        'font.family': 'Times New Roman',
        'font.weight': 'bold',
        'figure.figsize': (10, 10)  # 设置图像尺寸为10x10英寸
    })
contour = plt.contourf(X, Y, Z, levels=100, cmap='rainbow')
cbar = plt.colorbar(contour)
cbar.set_label('Cs (mAh/g)', rotation=270, labelpad=48, fontweight='bold', fontsize=32)

# 设置轴标签和标题
plt.xlabel('N', fontweight='bold', fontsize=32)
plt.ylabel('O', fontweight='bold', fontsize=32)
plt.title('Specific capacity prediction contour plot', fontweight='bold', fontsize=48)
plt.savefig('output/figures/contour_N&O.png', bbox_inches='tight', transparent=True, dpi=300)
plt.show()


In [ ]:
fixed_values = {
    "O": 8,
    "N": 2.5,
    # "Specific surface area": 1667.3,
    "Pore volume": 1.2,
    "Dap": 2.508,
    "ID/IG": 1.02,
    # "micro%": 0.48,
    "Current density": 0.2
}

# 生成等距分布的Dap和ID/IG的值
SSA_values = np.linspace(200, 3600, 100)
ratio_values = np.linspace(0, 1, 100)

columns = [
    "O", "N", "Specific surface area", "Pore volume", "Dap", "ID/IG", 
     "Current density", "micro%", 
]

# 创建一个空的DataFrame
data = []

# 填充数据
for SSA in SSA_values:
    for ratio in ratio_values:
        row = fixed_values.copy()
        row["Specific surface area"] = SSA
        row["micro%"] = ratio
        data.append(row)

df = pd.DataFrame(data, columns=columns)
df

In [ ]:
y_generated_pred = xgb.predict(df)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import griddata

# 生成示例数据
# 这部分需要替换为你的实际数据
feature_1 = df['Specific surface area']
feature_2 = df['micro%']
c = y_generated_pred

# 创建网格数据
x = np.linspace(200, 3600, 10)
y = np.linspace(0, 1, 10)
X, Y = np.meshgrid(x, y)

# 插值
Z = griddata((feature_1, feature_2), c, (X, Y), method='cubic')

# 绘制等高线图
plt.figure(figsize=(20, 19))
plt.rcParams.update({
        'font.family': 'Times New Roman',
        'font.weight': 'bold',
        'figure.figsize': (10, 10)  # 设置图像尺寸为10x10英寸
    })
contour = plt.contourf(X, Y, Z, levels=100, cmap='coolwarm')
cbar = plt.colorbar(contour)
cbar.set_label('Cs (mAh/g)', rotation=270, labelpad=48, fontweight='bold', fontsize=32)

# 设置轴标签和标题
plt.xlabel('Specific surface area', fontweight='bold', fontsize=32)
plt.ylabel('micro%', fontweight='bold', fontsize=32)
plt.title('Specific capacity prediction contour plot', fontweight='bold', fontsize=48)
plt.savefig('output/figures/contour_SSA&ratio.png', bbox_inches='tight', transparent=True, dpi=300)
plt.show()
